# Yolo-v1的pytorch实现

依照yolov1.cfg、yolov1原论文、[github上一份pytorch实现](https://github.com/ProgrammerZhujinming/YOLOv1/blob/main/YOLO_V1_Model.py)；

yolov1.cfg最后的fc层内容没太看懂；

module.py是方便fine-tune的版本；

In [17]:
import torch
from torch import nn
from torch.nn import functional as F
import sys
sys.path.append('../..')
import utils

测试填充函数：
- 输入的tensor：（batch，channel，height， width）
- 填充pad：[宽左，宽右，高上，高下]

In [18]:
x = torch.randn((1, 1, 4, 4))
x.shape, x

(torch.Size([1, 1, 4, 4]),
 tensor([[[[-0.8737, -0.3453, -0.0878, -0.3545],
           [-0.7838, -1.0773, -1.8497, -0.1728],
           [-1.2762,  0.7853,  1.8007,  1.3618],
           [-0.0407,  0.9483,  1.7208,  1.1484]]]]))

In [19]:
y = F.pad(x, pad=[1,2,3,4])
y.shape, y

(torch.Size([1, 1, 11, 7]),
 tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000, -0.8737, -0.3453, -0.0878, -0.3545,  0.0000,  0.0000],
           [ 0.0000, -0.7838, -1.0773, -1.8497, -0.1728,  0.0000,  0.0000],
           [ 0.0000, -1.2762,  0.7853,  1.8007,  1.3618,  0.0000,  0.0000],
           [ 0.0000, -0.0407,  0.9483,  1.7208,  1.1484,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]]]))

In [26]:
f = nn.ZeroPad2d([1,2,3,4])
y = f(x)
y.shape, y

(torch.Size([1, 1, 11, 7]),
 tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000, -0.8737, -0.3453, -0.0878, -0.3545,  0.0000,  0.0000],
           [ 0.0000, -0.7838, -1.0773, -1.8497, -0.1728,  0.0000,  0.0000],
           [ 0.0000, -1.2762,  0.7853,  1.8007,  1.3618,  0.0000,  0.0000],
           [ 0.0000, -0.0407,  0.9483,  1.7208,  1.1484,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]]]))

In [30]:
a = y[0, 0, 0, :4]
a

tensor([0., 0., 0., 0.])

虽然这个自适应padding形状的函数没用上

In [5]:
def compute_padding_shape(input_shape, kernel_size, stride):
    '''
    根据输入形状和步长，自动计算合适的padding_shape，以保持卷积过后，形状不变
    
    返回list[int]，[宽左，宽右，高上，高下]，为零填充的像素数
    '''
    padding=[]
    height, width = input_shape
    if isinstance(kernel_size, int):
        kernel_h, kernel_w = kernel_size, kernel_size
    else:
        kernel_h, kernel_w = kernel_size
    if isinstance(stride, int):
        stride_h, stride_w = stride, stride
    else:
        stride_h, stride_w = stride
    
    # 宽，(width - kernel_w + padding_w_left + padding_w_right) / stride_w + 1 == width
    padding.append(((width-1)*stride_w - width + kernel_w) / 2)             # 整除，取下整
    if ((width-1)*stride_w - width + kernel_w) % 2 == 0:
        padding.append(((width-1)*stride_w - width + kernel_w) / 2)
    else:
        padding.append(((width-1)*stride_w - width + kernel_w) / 2 + 1)
    # 高，也是类似
    padding.append(((height-1)*stride_h - height + kernel_h) / 2)
    if ((height-1)*stride_h - height + kernel_h) % 2 == 0:
        padding.append(((height-1)*stride_h - height + kernel_h) / 2)
    else:
        padding.append(((height-1)*stride_h - height + kernel_h) / 2 + 1)
    
    return padding

### 1、网络结构

网络中重复结构（卷积层）的构建函数，启发于VGG

In [6]:
def create_conv_layer(in_channels, out_channels, kernel_size, stride, padding, batch_normalize=1, activation='leaky'):
    '''
    构建yolov1中的卷积层，形如pad->conv2d->batch_norm->activation function
    
    Parameters:
    - in_channels, out_channels, kernel_size, stride, padding: 同nn.Conv2d
    - batch_normalize: 是否要做批量归一化，默认要做
    - activation: 激活函数种类，默认LeakyReLU
    '''
    layers = []
    layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding))
    if batch_normalize:
        layers.append(nn.BatchNorm2d(out_channels))
    if activation=='leaky':
        layers.append(nn.LeakyReLU(inplace=True))
    return layers

网络结构如图，图源自原论文，网络输入形如(batch_size, channel, height, width)

<img src='./figs/yolov1_model.png' />

其中7x7代表将图片均匀划分成7x7个grid cell，每个grid cell负责判断是否有物体的中心点落在上面：

<img src='./figs/yolov1_grid.png' style='zoom:50%'>

每个grid对应一个长度为（5xB + class num）的向量：

- B为每个grid负责回归的边界框个数；
- 每个grid只能取B个中最好的一个作为预测值，即网络最多只能预测SxS个物体，论文中S=7，这也是YOLO-V1对小物体、密集物体效果不好的原因；
- 5：每个边界框$(x_c, y_c, w, h, c)$，中心点位置、高宽、置信度；
- class num：后面跟的ont hot向量，表示预测为类别i的概率（有疑问）；

In [7]:
class Yolo_v1(nn.Module):
    '''
    Yolo-v1的pytorch实现
    
    输入形状为448x448，B为每个grid预测的个数，Class_Num为分类数
    '''
    def __init__(self, B=2, Classes_Num=20):
        super().__init__()
        # 提前padding，保持各个块的卷积过后，形状不变；
        # in:3x448x448, out:64x112x112
        self.blk1 = nn.Sequential(
            *create_conv_layer(3, 64, kernel_size=7, stride=2, padding=3),  # stride=2的conv和pool都是减高宽的
            # nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=2),
            # nn.BatchNorm2d(64),
            # nn.LeakyReLU(negative_slope=0.01),
            nn.MaxPool2d(2, stride=2)
        )
        # in:64x112x112, out:192x56x56
        self.blk2 = nn.Sequential(
            *create_conv_layer(64, 192, kernel_size=3, stride=1, padding=1),
            # nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(192),
            # nn.LeakyReLU(negative_slope=0.01),
            nn.MaxPool2d(2, stride=2)
        )
        # in:192x56x56, out:512x28x28
        self.blk3 = nn.Sequential(
            *create_conv_layer(192, 128, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(128, 256, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(256, 256, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(256, 512, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, stride=2)
        )
        # in:512x28x28, out:1024x14x14
        self.blk4 = nn.Sequential(
            *create_conv_layer(512, 256, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(256, 512, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(512, 256, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(256, 512, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(512, 256, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(256, 512, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(512, 256, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(256, 512, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(512, 512, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, stride=2)
        )
        # in:1024x14x14, out:1024x14x14
        self.blk4 = nn.Sequential(
            *create_conv_layer(1024, 512, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(512, 1024, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(1024, 512, kernel_size=1, stride=1, padding=0),
            *create_conv_layer(512, 1024, kernel_size=3, stride=1, padding=1)
        )
        # in:1024x14x14, out:1024x7x7
        self.blk5 = nn.Sequential(
            *create_conv_layer(1024, 1024, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(1024, 1024, kernel_size=3, stride=2, padding=1),
            *create_conv_layer(1024, 1024, kernel_size=3, stride=1, padding=1),
            *create_conv_layer(1024, 1024, kernel_size=3, stride=1, padding=1)
        )
        # in:7x7x1024, out:7x7x30
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(7 * 7 * 1024, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 7 * 7 * (B*5 + Classes_Num)),
            nn.Sigmoid()        # 增加sigmoid函数是为了将输出全部映射到(0,1)之间，因为如果出现负数或太大的数，后续计算loss会很麻烦
        )

    def forward(self, x):
        # 类GoogLeNet
        # 也可换成resnet50，删除stage5后的全局池化和全连接层，增加一个带有空洞卷积的网络block（模仿detnet网络，增加感受野而不改变输出特征图大小）
        x = self.blk1(x)
        x = self.blk2(x)
        x = self.blk3(x)
        x = self.blk4(x)
        # 后四层卷积
        x = self.blk5(x)
        # 为方便后处理，需要将输出构建成7x7x30，而不是30x7x7；为了符合直觉，需要做重排列将特征放在最后一维
        # (batch_size, channel, height, width) -> (batch_size, height, width, channel)
        x = x.permute(0, 2, 3, 1)
        # 展平+全连接层
        x = self.fc(x)
        # 重构
        x = x.view((-1, 7, 7, (self.B*5 + self.Classes_Num)))

    def initialize_params(self):
        '''网络参数初始化'''
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                m.bias.data.zero_()

疑问：(x,y,w,h,c)和p是否需要做softmax？？？

### 2、损失函数

Loss Function:

<img src='./figs/yolov1_loss.png' style='zoom:60%;' />

- 掩码：
  - $\mathbb{1}_{i}^{obj}$：有gt bbox的中心点落在grid $i$中；
  - $\mathbb{1}_{ij}^{obj}$：有gt bbox的中心点落在grid $i$中，且该grid回归的的B个bbox中，第$j$个bbox与该gt bbox的IoU最大；
  - $\mathbb{1}_{ij}^{noobj} == not \ (\mathbb{1}_{ij}^{obj})$；
- 第一项为中心点定位误差；
- 第二项为宽高定位误差：
  - 求根号可使小的bbox对误差更敏感；
- 第三、四项为confidence回归误差：
  - $\mathbb{1}_{ij}^{obj}==1$时（第三项），$\hat{C}_i$为该grid回归的第j个bbox，与中心点落在该grid的gt bbox的IoU；
  - $\mathbb{1}_{ij}^{obj}==0$时（第四项），$\hat{C}_i$为0；
  - $C_i$为预测值；
- 第五项为类别预测误差；
- 权重：
  - 物体检测问题，是一个典型的类别数目不均衡的问题。其中49个格点，含有物体的格点往往只有3、4个，其余全是不含有物体的格点。此时如果不采取点措施，那么物体检测的mAP不会太高，因为模型更倾向于不含有物体的格点。
  - $\lambda_{\mathbb{coord}}$：使网络更重视$(x_c,y_c,w,h)$的预测；
  - $\lambda_{noobj}$：对不负责预测obj的confidence loss，赋予小的权重；

In [ ]:
class Yolo_v1_loss(nn.Module):
    '''
    Yolo-v1的损失函数计算的pytorch实现

    Parameters:
    - img_shape: _size_2_t，[h, w]
    - S: _size_2_t，表示一张图片被划分成了[高x宽]个grid，默认为7表示7x7
    - B: int，一个grid回归多少个bbox
    - Class_num: int，类别个数
    - l_coord、l_noobj: 对应两个lambda
    '''
    def __init__(self, img_shape=448, S=7, B=2, Class_num=20, l_coord=5, l_noobj=0.5):
        super().__init__()
        # 图片的形状
        if isinstance(img_shape, int):
            self.img_h = img_shape
            self.img_w = img_shape
        elif isinstance(img_shape, list):
            self.img_h = img_shape[0]
            self.img_w = img_shape[1]
        # 网格的个数形状
        if isinstance(S, int):
            self.S_h = S
            self.S_w = S
        elif isinstance(S, list):
            self.S_h = S[0]
            self.S_w = S[1]
        self.grid_h = self.img_h / self.S_h     # 网格的高度和宽度
        self.grid_w = self.img_w / self.S_w
        self.B = B
        self.Class_num = Class_num
        self.l_coord = l_coord
        self.l_noobj = l_noobj

    def forward(self, pred, gt):
        '''
        前向计算
        - pred:  (batch_size, S_h, S_w, (B)*5+Class_num)，未经过预处理，仍然只是Yolo_v1输出的sigmoid
        - gt:    (batch_size, S_h, S_w, 5+Class_num)，已经经过预处理，框成center形式，其中confidence代表1_i^{obj}，类别成one-hot
        '''
        # 定义三个计算损失的变量 正样本定位损失 样本置信度损失 样本类别损失
        loss_coord = 0
        loss_confidence = 0
        loss_classes = 0
        iou_sum = 0
        object_num = 0

        mseLoss = nn.MSELoss()
        # pred的预处理
        for i in range(self.B):
            pred[:, :, :, i*5+0] = pred[:, :, :, i*5+0] * self.grid_w   # x_c，最大不能超过一个grid
            pred[:, :, :, i*5+1] = pred[:, :, :, i*5+1] * self.grid_h   # y_c
            pred[:, :, :, i*5+2] = pred[:, :, :, i*5+2] * self.img_w    # w，最大不能超过整张图片
            pred[:, :, :, i*5+3] = pred[:, :, :, i*5+3] * self.img_h    # h
        
        # loss计算
        # 其实也可以使用矩阵运算优化，但保险起见，还是用了循环
        for batch in range(len(pred)):
            for row in range(self.S_h):
                for col in range(self.S_w):
                    pred_ = pred[batch, row, col, :]
                    gt_ = gt[batch, row, col, :]
                    if gt_[4] == 0:
                        # [row, col]处的grid，没有gt框中心落在这，故B个框都是负样本，只需要加置信度loss
                        for i in range(self.B):
                            loss_confidence += self.l_noobj * torch.pow(pred_[i*5 + 4], 2)
                    else:
                        # [row, col]处的grid，有gt框中心落在这
                        object_num += 1
                        # 分别计算回归的B个框与gt的IoU
                        iou = torch.zeros(self.B)
                        gt_bbox = utils.bbox_center_to_corner(gt_[:4])
                        for i in range(self.B):
                            pred_bbox = utils.bbox_center_to_corner(pred_[i*5:(i*5+4)])
                            iou[i] = utils.IoU(pred_bbox, gt_bbox)
                        # 取IoU最大的成为正样本，其它B-1个成为负样本
                        val, idx = iou.topk(1)
                        # 定位loss
                        loss_coord += self.l_coord * (torch.pow(pred_[idx*5] - gt_[0], 2) + torch.pow(pred_[idx*5+1] - gt_[1], 2)) + \
                                      self.l_coord * (torch.pow(torch.sqrt(pred_[idx*5+2]) - torch.sqrt(gt_[2]), 2) + \
                                                      torch.pow(torch.sqrt(pred_[idx*5+3]) - torch.sqrt(gt_[3]), 2))
                        # 置信度loss
                        iou_sum += val
                        for i in range(self.B):
                            if i == idx:
                                # 正样本
                                loss_confidence += torch.pow(pred_[i*5 + 4] - val, 2)
                            else:
                                # 负样本
                                loss_confidence += self.l_noobj * torch.pow(pred_[i*5 + 4], 2)
                        # 类别预测loss
                        gt_class = gt_[5:]
                        pred_class = pred_[(self.B*5):]
                        loss_classes += mseLoss(gt_class, pred_class)
        loss_coord /= len(pred)
        loss_confidence /= len(pred)
        loss_classes /= len(pred)
        loss = loss_coord + loss_confidence + loss_classes
        return loss, loss_coord, loss_confidence, loss_classes, iou_sum, object_num

### 3、预测阶段后处理：非极大值抑制（NMS）

不应用于训练阶段，只应用于训练完成后，应用模型进行预测的阶段。

非极大值抑制（Non-Maximum Suppression）：